# Summary
This document corresponds to Exercise 2 of [this file](https://github.com/PerformanceEstimation/Learning-Performance-Estimation/blob/main/Exercises/Course.pdf).

We first install a few packages.

In [ ]:
!pip install sympy
!pip install cvxpy

## Closed-form solutions for the LMI?

Let us start by trying to solve this LMI numerically for a few values of $\mu$, $L$, and $\gamma_k$.
The following code implements the LMI.

### Numerical solutions to the LMI

In [ ]:
import cvxpy as cp
def lmi_convergence_distance(L, mu, gamma):

    # Write the LMI.
    S = cp.Variable((2, 2)) # this is the matrix S
    l1 = cp.Variable()      # this is the multiplier (lambda1 == lambda2) which we denote l1
    tau = cp.Variable()     # this is the objective
    
    s11 = tau-1+l1*L*mu/(L-mu)
    s12 = gamma-l1*(L+mu)/2/(L-mu)
    s22 = l1/(L-mu)-gamma**2
    
    constraints = [S >> 0,
                   S[0,0] == s11,
                   S[1,1] == s22,
                   S[0,1] == s12,
                   S[1,0] == s12,
                   l1 >= 0]
    
    prob = cp.Problem(cp.Minimize(tau), constraints)
    prob.solve()
    return tau.value, l1.value, S.value

The following code solves the LMI for a grid of $\gamma_k$'s.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

nb_test = 50
mu = .1
L = 1
gamma = np.linspace(-1., 3., num=nb_test)

l1_list = list()
tau_list = list()
S_list = list()

for i in range(nb_test):
    tau,l1,S = lmi_convergence_distance(L, mu, gamma[i])
    l1_list.append(l1[()])
    tau_list.append(tau[()])
    S_list.append(S)

In [ ]:
plt.plot(gamma, tau_list, '.-',label='$\tau_1$')
plt.plot(gamma, l1_list, '.-',label='$\lambda_1$')

plt.xlabel('$\gamma$')
plt.ylabel('Multipliers')
plt.legend()
plt.show()

A useful information about $S$ is its eigenvalues.

In [ ]:
firsteig_list = list()
seceig_list = list()

for i in range(nb_test):
    eigsV, _ = np.linalg.eigh(S_list[i])
    firsteig_list.append(np.max(eigsV))
    seceig_list.append(np.min(eigsV))

plt.plot(gamma, firsteig_list, '.-',label='first eigenvalue')
plt.plot(gamma, seceig_list, '.-',label='second eigenvalue')

plt.xlabel('$\gamma$')
plt.ylabel('$\Lambda(S)$')
plt.legend()
plt.show()

In case you have a candidate expression for $\tau(\mu,L,\gamma_k)$, compare it with the numerics?

In [ ]:
tau_candidates = [ np.max([(1-gamma[i]*L)**2,(1-gamma[i]*mu)**2]) for i in range(nb_test)]
plt.plot(gamma, tau_list, '.-',label='LMI')
plt.plot(gamma, tau_candidates, '--',label='TRIAL')

plt.xlabel('$\gamma$')
plt.ylabel('Multipliers')
plt.legend()

plt.show()

### Closed-form solutions to the LMI?

As we saw numerically, the matrix $S$ has rank $1$ for most values of the step-sizes!
As the problem of finding a bound on the convergence rate $\frac{\|x_{k+1}-x_\star\|^2}{\|x_k-x_\star\|^2}$ corresponds to a linear problem with an LMI constraint, it is natural that the optimal solution is on the boundary of the feasible set, and we can use that for solving the problem in closed-form.

We use a bit of symbolic computation below for simplicity.

Complete the following code with the values in $S$:

In [ ]:
import sympy as sm

tau = sm.Symbol('tau')
l1 = sm.Symbol('l1')
gamma = sm.Symbol('gamma')
L = sm.Symbol('L')
mu = sm.Symbol('mu')

S = sm.Matrix([[tau-1+l1*L*mu/(L-mu), gamma-l1*(L+mu)/2/(L-mu)], [gamma-l1*(L+mu)/2/(L-mu), l1/(L-mu)-gamma**2]])

For making $S$ rank defficient, let's choose $\tau$ for cancelling out the determinant:

In [ ]:
candidate_tau=sm.solve(S.det(),tau)
candidate_tau[0]

There are two possibilities for choosing $\lambda_1$:
- on the boundary of the PSD cone ($S=0$),
- minimize $\tau$ (and verify that the corresponding $(\lambda_1,\tau,S)$ is feasible for the LMI afterwards).

Because we observed numerically that $S$ was rank $1$ for most choices of stepsizes, let's focus on the second possibility.

In [ ]:
dtau=sm.simplify(sm.diff(candidate_tau[0],l1)) #differentiate tau with respect to lambda_1
dtau

In [ ]:
l1sol=sm.solve(dtau,l1) # solve dtau/dlambda_1 == 0 in lambda1
l1sol # show the two possibilities!

The corresponding "final" expressions for $\tau$ that must be checked are therefore:

In [ ]:
exprtau1=sm.simplify(candidate_tau[0].subs(l1,l1sol[0]))
exprtau1.factor()

In [ ]:
exprtau2=sm.simplify(candidate_tau[0].subs(l1,l1sol[1]))
exprtau2.factor()

### Bonus: how to get to the LMI using symbolic computations?

In [ ]:
x0 = sm.Symbol('x0')
g0 = sm.Symbol('g0')
f0 = sm.Symbol('f0')
xs = 0 # wlog, x_* = 0
gs = 0 # constraint g_* = 0
fs = 0 # wlog, f_* = 0
gamma = sm.Symbol('gamma')
x1 = x0 - gamma * g0

L = sm.Symbol('L')
mu = sm.Symbol('mu')

tau = sm.Symbol('tau')
l1 = sm.Symbol('l1')
l2 = sm.Symbol('l2')
# the two interpolation constraints in the form "constraint <= 0"
constraint1 = f0 - fs + g0*(xs-x0) + 1/2/L * g0**2 + mu/(2*(1-mu/L)) * (x0-xs-1/L*g0)**2
constraint2 = fs - f0 + 1/2/L * g0**2 + mu/(2*(1-mu/L)) * (x0-xs-1/L*g0)**2
primal_objective = (x1-xs)**2
initial_condition = (x0-xs)**2

# This LMI should be >= 0
LMI = sm.simplify(sm.hessian( l1*constraint1 + l2*constraint2 + tau*initial_condition - primal_objective, (x0,g0))/2) 
# This is the linear constraint ==0 in the LMI
LinearConst = sm.simplify(sm.diff(l1*constraint1 + l2*constraint2 + tau*initial_condition - primal_objective,f0))

# For getting the same LMI as in the document, substitute l2 by l1 and simplify
LMI_simplified = sm.simplify(LMI.subs(l2,l1))

In [ ]:
LMI_simplified